## 1. Import Libraries

In [ ]:
! pip install openai
! pip install langchain
! pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.1.1
    Uninstalling pydantic-2.1.1:
      Successfully uninstalled pydantic-2.1.1


In [ ]:
import os
import openai
from langchain.chat_models import ChatOpenAI

## 2. Import Credentials

In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## 3. LLM call using LangChain

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
chat = ChatOpenAI(temperature=0.0)
chat

## 4. Create Prompt Template

In [ ]:
{"area":"Almada",
 "type_room":"double",
 "price_value":500,
 "availability_from":"Aug",
 "number_months":6
}

In [ ]:
rent_annoucement = """
Room available with river view! Almada.
Great room, close to Cristo Rey, amazing kitchen, River view, close to metro and train and bus, available for 1 or 2 months max. (aug - oct)
Pet friendly, all services included (Wifi, washing machine, water and electriciy)
475 euros per month. (room only for 1 person), you will be sharing the house only with one more room with my wife, me and our puppy
"""

In [ ]:
rent_template = """\
For the following text of , extract the following information:

area: where the apartment is located

type_room: Is the room single or double? Answer single or double

number_roomates: number of roomates

price_value: Extract value of the price of rented room

availability_from: when the room is available

number_months: number of month of availability of the room

Format the output as JSON with the following keys:
area
type_room
number_roomates
price_value
availability_from
number_months

text: {text}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(rent_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text of , extract the following information:\n\narea: where the apartment is located\n\ntype_room: Is the room single or double? Answer single or double\n\nnumber_roomates: number of roomates\n\nprice_value: Extract value of the price of rented room\n\navailability_from: when the room is available\n\nnumber_months: number of month of availability of the room\n\nFormat the output as JSON with the following keys:\narea\ntype_room\nnumber_roomates\nprice_value\navailability_from\nnumber_months\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [ ]:
messages = prompt_template.format_messages(text=rent_annoucement)
response = chat(messages)
print(response.content)

{
  "area": "Almada",
  "type_room": "single",
  "number_roomates": 3,
  "price_value": 475,
  "availability_from": "aug",
  "number_months": 2
}


In [ ]:
type(response.content)

str

## 5. Add Schema

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [ ]:
area_schema = ResponseSchema(name="area",
                             description="where the apartment is located")
type_room_schema = ResponseSchema(name="type_room",
                                      description="Is the room single or double? Answer single or double")
number_roomates_schema = ResponseSchema(name="number_roomates",
                                    description="number of roomates in the appartment or in the house")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract value of the price of rented room")

availability_from_schema = ResponseSchema(name="price_value",
                                    description="when the room is available")

number_months_schema = ResponseSchema(name="number_months",
                                    description="number of months of availability of the room")

response_schemas = [area_schema, type_room_schema, number_roomates_schema, price_value_schema, availability_from_schema, number_months_schema]

In [ ]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"area": string  // where the apartment is located
	"type_room": string  // Is the room single or double? Answer single or double
	"number_roomates": string  // number of roomates in the appartment or in the house
	"price_value": string  // Extract value of the price of rented room
	"price_value": string  // when the room is available
	"number_months": string  // number of months of availability of the room
}
```


In [ ]:
rent_annoucement = """
Room available with river view! Almada.
Great room, close to Cristo Rey, amazing kitchen, River view, close to metro and train and bus, available for 1 or 2 months max. (aug - oct)
Pet friendly, all services included (Wifi, washing machine, water and electriciy)
475 euros per month. (room only for 1 person), you will be sharing the house only with one more room with my wife, me and our puppy
"""

In [ ]:
rent_template_2 = """\
For the following text of , extract the following information:

area: where the apartment is located

type_room: Is the room single or double? Answer single or double

number_roomates: number of roomates in the appartment or in the house

price_value: Extract value of the price of rented room

availability_from: when the room is available

number_months: number of months of availability of the room

Format the output as JSON with the following keys:
area
type_room
number_roomates
price_value
availability_from
number_months

text: {text}

{format_instructions}
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template=rent_template_2)

messages = prompt.format_messages(text=rent_annoucement,
                                format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

In [ ]:
response = chat(messages)
print(response.content)

```json
{
	"area": "Almada",
	"type_room": "single",
	"number_roomates": "3",
	"price_value": "475 euros per month",
	"availability_from": "aug",
	"number_months": "2"
}
```


In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

{'area': 'Almada',
 'type_room': 'single',
 'number_roomates': '3',
 'price_value': '475 euros per month',
 'availability_from': 'aug',
 'number_months': '2'}

In [ ]:
type(output_dict)

dict

In [ ]:
output_dict.get('number_months')

'2'

### Other example

In [ ]:
rent_annoucement_2 = """
AVAILABLE ROOM IN CARCAVELOS
DATE: 1st July 2023 - 30th July 2023
PRICE: 465€
Refurnished apartment with capacity for 6 people, with 2 shared bathrooms and 1 private bathroom, located in Oeiras, only 7 minutes walking to Nova SBE campus in Carcavelos, 11 minutes walking from Oeiras train station and around 20 minutes walking to Carcavelos beach. From Oeiras train station you will take around 25 minutes to arrive to Cais de Sodré, Lisbon. The apartment is located on lower 4th floor, in a safe and quite neighbourhood, surrounded by green parks. Bus station is located 2 minutes walking and the nearest supermarket available is Pingo Doce, only 7 minutes walking. The apartment is decorated with a sober nordic style, is fully equipped and has all amenities you need. Ideal for someone looking for proximity to NOVA SBE, NATO or to the beach. No elevator. Includes expenses up to 40 monthly. Only July available. Every extra day in August 60€
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template=rent_template_2)

messages = prompt.format_messages(text=rent_annoucement_2,
                                format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

For the following text of , extract the following information:

area: where the apartment is located

type_room: Is the room single or double? Answer single or double

number_roomates: number of roomates in the appartment or in the house

price_value: Extract value of the price of rented room

availability_from: when the room is available

number_months: number of months of availability of the room

Format the output as JSON with the following keys:
area
type_room
number_roomates
price_value
availability_from
number_months

text: 
AVAILABLE ROOM IN CARCAVELOS
DATE: 1st July 2023 - 30th July 2023
PRICE: 465€
Refurnished apartment with capacity for 6 people, with 2 shared bathrooms and 1 private bathroom, located in Oeiras, only 7 minutes walking to Nova SBE campus in Carcavelos, 11 minutes walking from Oeiras train station and around 20 minutes walking to Carcavelos beach. From Oeiras train station you will take around 25 minutes to arrive to Cais de Sodré, Lisbon. The apartment is loca

In [ ]:
response = chat(messages)
print(response.content)

```json
{
	"area": "Carcavelos",
	"type_room": "single",
	"number_roomates": "6",
	"price_value": "465€",
	"availability_from": "1st July 2023",
	"number_months": "1"
}
```


In [ ]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

{'area': 'Carcavelos',
 'type_room': 'single',
 'number_roomates': '6',
 'price_value': '465€',
 'availability_from': '1st July 2023',
 'number_months': '1'}

In [ ]:
type(output_dict)

dict

In [ ]:
output_dict.get('number_months')

'1'